In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import mutagen
import os
import random
import wave
import numpy as np
from scipy.io.wavfile import read as read_wav
from scipy.io.wavfile import write as write_wav
import librosa
import librosa.display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

import utils
import classifier
import multi_classifier as mclass
import irmasTestUtils as testUtils

In [3]:
df_train = utils.load_pickled_dataset("preprocessed_train.pkl")
df_test = utils.load_pickled_dataset("preprocessed_test.pkl")

In [ ]:
#x, ys = mclass.process_dataframe(df_train)

In [ ]:
#model = mclass.make_model()
#mclass.train_model(model, x, ys, batch_size=32, epochs=30)

In [4]:
df_train = df_train.sample(frac=1)

In [ ]:
model = classifier.make_model()

classifier.train_model(model, df_train, batch_size=32, epochs=30)


Epoch 1/30
420/420 [==============================] - 22s 50ms/step - loss: 6.3694
Epoch 2/30
420/420 [==============================] - 24s 57ms/step - loss: 1.6948
Epoch 3/30
420/420 [==============================] - 26s 62ms/step - loss: 1.4144 0s -
Epoch 4/30
420/420 [==============================] - 26s 62ms/step - loss: 1.0495
Epoch 5/30
420/420 [==============================] - 26s 62ms/step - loss: 0.7051
Epoch 6/30
420/420 [==============================] - 28s 67ms/step - loss: 0.4278
Epoch 7/30
420/420 [==============================] - 29s 68ms/step - loss: 0.2502
Epoch 8/30
420/420 [==============================] - 27s 64ms/step - loss: 0.1446
Epoch 9/30
420/420 [==============================] - 31s 74ms/step - loss: 0.0963
Epoch 10/30
420/420 [==============================] - 30s 71ms/step - loss: 0.1405
Epoch 11/30
420/420 [==============================] - 26s 62ms/step - loss: 0.0668
Epoch 12/30
420/420 [==============================] - 26s 63ms/step - loss: 0.2

In [ ]:
print(classifier.get_accuracy(model, df_test))

In [ ]:

print(model.weights)
model.save_weights("classifier_weights")

In [ ]:
test = classifier.make_model()
test.load_weights("classifier_weights")

In [ ]:



smaller = mclass.make_model()
for layer in test.layers:
    for i in range(len(smaller)):
        for v in range(len(smaller[i].layers)):
            if smaller[i].layers[v].name == layer.name:
                smaller[i].layers[v].set_weights(layer.get_weights())

In [ ]:
print(smaller[0].layers)

In [ ]:
x, ys = mclass.process_dataframe(df_train)

In [ ]:
mclass.train_model(smaller, x, ys, batch_size=32, epochs=50)

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=4, strides=(2,2), padding='same', input_shape=(128,259,1), name="conv_1"))
model.add(tf.keras.layers.BatchNormalization(axis=1, name="bn_1"))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="mpool_1"))

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=(1,1), padding='valid', name="conv_2"))
model.add(tf.keras.layers.BatchNormalization(axis=1, name="bn_2"))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="mpool_2"))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, strides=(1,1), padding='valid', name="conv_3"))
model.add(tf.keras.layers.BatchNormalization(axis=1, name="bn_3"))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), name="mpool_3"))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(500, activation='relu', name="fc_4"))
model.add(tf.keras.layers.Dense(11, activation=None, name="fc_5"))
model.add(tf.keras.layers.Softmax())

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy())

train_set = df.tail(-1000)
test_set = df.head(1000)

x, y = np.stack(train_set.data), np.stack(train_set.label)
x = x.reshape((-1,128,259,1))
print(x.shape)
print(x.shape, y.shape)

model.fit(x, y, batch_size=32, epochs=30)

'''x, y = np.stack(sample['data']), np.stack(sample['label'])
print(x.shape, y.shape)
print(type(x), type(y))
x, y = tf.convert_to_tensor(x, dtype=tf.float32), tf.convert_to_tensor(y, dtype=tf.float32)
model.predict(x)'''

In [ ]:
x, y = np.stack(train_set.data), np.stack(train_set.label)
x = x.reshape((-1,128,259,1))

preds = np.argmax(model.predict(x), axis=1).reshape(x.shape[0], 1)
true = np.argmax(y, axis=1)

print("Train accuracy:", np.sum((preds == true)*np.ones(preds.shape))/x.shape[0])

In [ ]:
x, y = np.stack(test_set.data), np.stack(test_set.label)
x = x.reshape((-1,128,259,1))

preds = np.argmax(model.predict(x), axis=1).reshape(x.shape[0], 1)
true = np.argmax(y, axis=1)

print("Test accuracy:", np.sum((preds == true)*np.ones(preds.shape))/x.shape[0])

In [ ]:
#15 epochs - train 88.0 test 51.5


#30 epochs - train accuracy 96.7%, test accuracy 36.8%
#20 epochs - train accuracy 98.8%, test accuracy 47.8%
#15 epochs - train accuracy 97.5%, test accuracy 36.4%
#10 epochs - train accuracy 95.7%, test accuracy 44.4%

arr = np.array([[5, 2], [3, 4], [6, 8]])
print (arr)
print (np.linalg.norm(arr))
print (arr / np.linalg.norm(arr))

In [ ]:
utils.combine_dataset(df, 5, 3)